In [ ]:
# Task 1

In [ ]:
# Task 2

In [ ]:
# Task 3.1
# File is in the folder

In [16]:
# Task 3.2
import csv
import sqlite3

with open('sanitisers.txt') as f:
    reader = csv.reader(f)
    reader = list(reader)[1:]

connection = sqlite3.connect('sanitisers.db')

cursor = connection.cursor()

#Note that 'alcohol-based' field contains a special character '-' so we need to surround it with quotation marks to force it to recognize it as a field name. That said, this is probably a bad practice for the field name
insert_sql = "INSERT INTO sanitisers(product_name,active_ingredient,'alcohol-based') VALUES(?,?,?)"

product_name = 0
active_ingredient = 1
alcohol_based = 2

data_tuples = [ (product[0],product[1],product[2]) for product in reader]

for data in data_tuples:
    cursor.execute(insert_sql,data)

connection.commit()

cursor.close()

In [4]:
# Task 3.3

from flask import Flask, render_template, request

import sqlite3

app = Flask(__name__)

@app.route('/')
def index():
    connection = sqlite3.connect('sanitisers.db')
    cursor = connection.cursor()
    products =  list(cursor.execute('SELECT * FROM sanitisers'))
    return render_template('index.html',products=products)

@app.route('/active_ingredient',methods=['GET'])
def form():
    #Note that we're using request.values() here instead of request.form.get()
    #Alternatively, we can use request.args.get() for GET (query string) value
    active_ingredient = request.values.get('active_ingredient')
    connection = sqlite3.connect('sanitisers.db')
    cursor = connection.cursor()
    #The following approach works, but is vulnerable to SQL injection
    # myQuery = f'SELECT * FROM sanitisers WHERE active_ingredient = "{active_ingredient}"'
    # products =  list(cursor.execute(myQuery))
    
    #It's better to use the parametric substituion method to not get SQL injected
    #Also note the 2nd argument in the cursor.execute() method is a tuple, and thus, we include a comma ',' in the parantheses
    products =  list(cursor.execute('SELECT * FROM sanitisers WHERE active_ingredient = ?',(active_ingredient,)))

    return render_template('filtered.html',products=products)

app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jan/2022 13:27:30] "GET /active_ingredient?active_ingredient=Triclosan HTTP/1.1" 200 -


[('Soapy Sudz', 'Triclosan', 'No'), ('Anti-Bac Foam Soap', 'Triclosan', 'No'), ('Acclaim', 'Triclosan', 'No'), ('Hygenipak Plus Anti-Microbial Skin Cleanser', 'Triclosan', 'No'), ('Adasept Liq 0.5%', 'Triclosan', 'No'), ('Winning Hands Foaming Antibacterial Hand Cleanser', 'Triclosan', 'No'), ('Biotech-Transfer Skin Shield Barrier Lotion', 'Triclosan', 'No'), ('Bacti-Stat 0.3%', 'Triclosan', 'No'), ('LEVER 2000 Antibacterial Liquid Soap', 'Triclosan', 'No'), ('Savon Antiseptique Milan', 'Triclosan', 'No'), ('Zep Tranquil Meadows', 'Triclosan', 'No'), ('Applaud', 'Triclosan', 'No'), ('Antiseptic Hand Soap Liq 0.3%', 'Triclosan', 'No'), ('Florafree Gel 0.3%', 'Triclosan', 'No'), ('Asept', 'Triclosan', 'No'), ('Bacti-Stat Ae', 'Triclosan', 'No'), ('Orasoft 3', 'Triclosan', 'No'), ('Crown Antiseptic Hand Soap', 'Triclosan', 'No'), ('Valour Antiseptic Hand Cleanser', 'Triclosan', 'No'), ('Soft Care Nutra-Germ Antibacterial Lotion Soap', 'Triclosan', 'No'), ('Antibacterial Lotion Soap', 'Tri

In [3]:
# Task 4.1
from pymongo import MongoClient

import json

with open('balloons.json') as f:
    balloon_items = json.load(f)

try:
    client = MongoClient('localhost', 27017)
    print("Connected successfully!!!")
except:
    print("Could not connect to MongoDB")

db = client['products']

coll = db['balloons']

coll.insert_many(balloon_items)

Connected successfully!!!


In [5]:
# Task 4.2
myQuery = {'design':'car'}

for doc in coll.find(myQuery):
    print(doc['amount'])

88


In [8]:
# Task 4.3
myQuery = {'helium':None}

myUpdate = {'$set':{'helium','no'}}

coll.update_many(myQuery,myUpdate)

In [13]:
# Task 4.4
myQuery =   {'$and':
                    [   {'helium':'no'},
                        {'$and':[
                                    {'colours':{'$in':['red','yellow','blue','green']}},
                                    {'colours':{'$nin':['black']}}
                                ]
                        }
                    ]
            }

for doc in coll.find(myQuery):
    print(doc['design'])



car
cloud
